# 프롬프트 엔지니어링 기법
https://www.promptingguide.ai/kr/techniques

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

### Zero-shot Prompting
: 예시가 주어지지 않음

In [2]:
prompt = '''
다음 영어 문장을 불어로 번역해줘?
'Hello, how are you?'
'''

completion = client.chat.completions.create(
    model = "gpt-4o-mini" ,
    temperature= 0.2,
    messages = [ {"role":"user",
                  "content":prompt}
               ]
)

# 응답 출력
print(completion.choices[0].message.content)

'Hello, how are you?'는 프랑스어로 'Bonjour, comment ça va ?'입니다.


### Few-shot Prompting 
: 몇 개의 예시를 제공 , 한 개의 예시가 주어지는 경우 --> One-shot

In [7]:
prompt = """
English: 'Hello, how are you?'
French: 'Bonjour, comment ça va?'

English: 'What is your name?'
French: 'Comment vous appelez-vous?'

English: 'Hello,Miss'
English:
"""

completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)

*French: 'Bonjour, mademoiselle'


In [17]:
prompt = """
아래 예시를 참조해서 알맞은 답변해줘

질문: '이 영화 너무 재미 없어!!'
답변: 부정 평가

질문: '이 영화 돈이 안 아까와..'
답변: 긍정 평가

질문: '시간만 버렸어~'
답변: 부정 평가

질문: '내내 하품만 나옴'
답변: 

질문: '핵 꿀잼~'
답변: 
"""

completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)

질문: '내내 하품만 나옴'  
답변: 부정 평가

질문: '핵 꿀잼~'  
답변: 긍정 평가


In [20]:
# One-shot Promptinig
prompt = """
아래 예시를 참조해서 알맞은 답변해줘

질문: '이 영화 너무 재미 없어!!'
답변: 부정 평가

질문: '내내 하품만 나옴'
답변: 

질문: '핵 꿀잼~'
답변: 

질문: '이 영화 돈이 아깝지 않다..'
답변: 
"""

completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)

질문: '내내 하품만 나옴'  
답변: 부정 평가

질문: '핵 꿀잼~'  
답변: 긍정 평가

질문: '이 영화 돈이 아깝지 않다..'  
답변: 긍정 평가


### CoT(Chain-of-Thought) Prompting

In [27]:
prompt = '''
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요?
'''
completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content) # 7개, 정답

철수의 사과 개수를 계산해보겠습니다.

1. 처음에 철수는 사과가 10개 있었습니다.
2. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다. 오늘은 3일째이므로:
   - 2개 × 3일 = 6개를 먹었습니다.
3. 따라서 오늘 아침까지 남은 사과는:
   - 10개 - 6개 = 4개입니다.
4. 오늘 점심에 어머니께서 사과 5개를 더 사다 주셨으므로:
   - 4개 + 5개 = 9개입니다.
5. 오늘 저녁에 친구에게 2개를 주었으므로:
   - 9개 - 2개 = 7개입니다.

따라서 철수는 총 7개의 사과를 가지고 있습니다.


In [33]:
prompt = '''
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요?
'''
completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content) # 11개 or 9개: 틀린 답변

철수는 현재 11개의 사과를 가지고 있습니다. 처음에 10개의 사과를 가졌고, 매일 아침 2개씩 먹어서 6개가 줄었으며, 어머니께서 5개를 더 사다 주셨기 때문에 11개가 되었습니다. 이후 친구에게 2개를 주었으므로 11 - 2 = 9개가 남게 됩니다.


In [30]:
prompt = '''
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요? 단계별로 분리해서 생각해 보세요
'''
completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)  # 7개 : 정답(틀릴 경우도 있다, 9개)

1. 처음에 가지고 있던 사과의 수: 10개
2. 매일 아침에 먹은 사과의 수: 2개 * 3일 = 6개
3. 어머니가 사다 준 사과의 수: 5개
4. 친구에게 준 사과의 수: 2개

따라서, 철수가 현재 가지고 있는 사과의 수는
10개 (처음에 가지고 있던 수) - 6개 (아침에 먹은 수) + 5개 (어머니가 사다 준 수) - 2개 (친구에게 준 수) = 7개

따라서, 철수는 현재 7개의 사과를 가지고 있습니다.


In [36]:
prompt="""
1. 철수에게는 사과가 10개 있었습니다.
2. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다.
   (3일 동안 2개씩 먹어서 2 * 3 = 6개의 사과를 먹었습니다.)
3. 오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다.
4. 오늘 저녁에 친구에게 2개를 주었습니다.
철수는 총 몇 개의 사과를 가지고 있을까요?
"""

completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)  # 7개 : 정답

철수는 현재 7개의 사과를 가지고 있습니다. 처음에 10개의 사과를 가졌고, 6개를 먹었으며, 어머니께서 5개를 더 사다 주었고, 친구에게 2개를 주었기 때문에 10 - 6 + 5 - 2 = 7개의 사과가 남게 됩니다.


In [39]:
prompt='''
집합 {4, 8, 9, 15, 12, 2, 1}에서 홀수를 모두 더하면 짝수야.
답변: 홀수(9, 15, 1)를 모두 더하면 25가 돼. 위의 명제는 거짓이야.

집합 {17, 10, 19, 4, 8, 12, 24}에서 홀수를 모두 더하면 짝수야.
답변: 홀수(17, 19)를 모두 더하면 36이 돼. 위의 명제는 참이야.

집합 {16, 11, 14, 4, 8, 13, 24}에서 홀수를 모두 더하면 짝수야.
답변: 홀수(11, 13)를 모두 더하면 24가 돼. 위의 명제는 참이야.

집합 {17, 9, 10, 12, 13, 4, 2}에서 홀수를 모두 더하면 짝수야.
답변: 홀수(17, 9, 13)를 모두 더하면 39가 돼. 위의 명제는 거짓이야.

집합 {15, 32, 5, 13, 82, 7, 1}에서 홀수를 모두 더하면 짝수야.
답변:
'''
completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)  # 정

홀수(15, 5, 13, 7, 1)를 모두 더하면 41이 되므로 위의 명제는 거짓입니다.


### SC(Self-Consistency) Prompting
: 여러번 질의해서 가장 빈도가 높은 답을 선택한다

In [51]:
from collections import Counter
def generate_responses(prompt, n=5):
    responses = []
    for _ in range(n):
        response =  client.chat.completions.create(
            # model="gpt-4o-mini",
            model="gpt-3.5-turbo",
            messages=[ {"role": "user",  "content": prompt}],
            temperature=0.7,  # 다양성 확보를 위해 온도를 높게 설정
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        responses.append(response.choices[0].message.content)
    return responses

# 문제 정의
prompt = """
철수에게는 사과가 10개 있었습니다. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다.
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다.
오늘 저녁에 친구에게 2개를 주었습니다.
철수는 총 몇 개의 사과를 가지고 있을까요?
"""
# 여러 번 답변 생성
responses = generate_responses(prompt,5)

In [52]:
# 가장 자주 등장하는 답변을 선택
# response_count.most_common() 메서드는 Counter 객체에 저장된 요소들의 
# 빈도수를 기준으로 정렬된 리스트를 반환,  [0][0]는 첫번째 요소 선택
def select_most_consistent_response(responses):
    response_count = Counter(responses)
    # print(type(response_count))
    most_common_response = response_count.most_common(1)[0][0]
    # print(response_count.most_common(3))
    return most_common_response

final_answer = select_most_consistent_response(responses)

# 결과 출력
print("Generated Responses:")
for i, response in enumerate(responses, 1):
    print(f"Response {i}: {response}")
    print('-'*100)

print("\nMost Consistent Answer:")
print(final_answer)

Generated Responses:
Response 1: 철수는 현재 11개의 사과를 가지고 있습니다. 처음에 10개를 가졌고, 매일 아침에 2개씩 먹었으므로 2일 동안 4개를 먹었습니다. 따라서 10 - 4 = 6개가 남았습니다. 어머니께서 5개를 더 사다주었으므로 6 + 5 = 11개의 사과를 가지고 있습니다. 이후 친구에게 2개를 주었으므로 11 - 2 = 9개가 남았습니다. 따라서 철수는 현재 9개의 사과를 가지고 있습니다.
----------------------------------------------------------------------------------------------------
Response 2: 철수는 오늘까지 총 10개의 사과를 가지고 있었고, 그저께부터 오늘까지 매일 2개씩 먹었으므로 3일 동안 총 6개를 먹었습니다.
따라서 오늘 점심에 어머니께서 사과 5개를 더 사다 주었으므로 10 + 5 = 15개가 되었습니다.
하지만 오늘 저녁에 친구에게 2개를 주었으므로 15 - 2 = 13개의 사과를 가지고 있습니다. 따라서 철수는 총 13개의 사과를 가지고 있습니다.
----------------------------------------------------------------------------------------------------
Response 3: 철수가 가지고 있는 사과의 개수는 11개입니다. 처음에 10개를 가졌고, 매일 아침 2개씩 먹어서 4개가 줄었으며, 어머니가 5개를 더 사다 주었기 때문에 11개가 됩니다. 이후 친구에게 2개를 주어 9개가 남습니다.
----------------------------------------------------------------------------------------------------
Response 4: 철수는 현재 (10개 - 6개(2개씩 먹은 날 수) + 5개(어머니가 사다 주신 개수) - 2개(친구에게 준 개수)) = 7개의 사과를 가지고 있습니

In [53]:
# * ML 앙상블 모델
# [1] Voting : (1) Soft Voting (2) Hard Voting : 다수결
# [2] Bagging
# [3] Boosting

In [63]:
from collections import defaultdict
import re

prompt = """
4명의 생존자가 강을 건너야 합니다. 각 생존자가 강을 건너는 데 걸리는 시간은 각각 다릅니다: 3분, 4분, 7분, 12분.
이들은 한 번에 최대 두 명씩만 함께 배를 타고 강을 건널 수 있으며, 배에는 반드시 등불을 지참해야 합니다. 
그러나 등불은 하나뿐이어서, 생존자들이 강을 건널 때마다 등불을 반드시 함께 가져가야 합니다. 
모든 생존자가 가장 효율적으로 강을 건너려면 어떻게 해야 할까요? 
마지막으로 총 소요 시간을 '정답: <정답>분' 형식으로 답변하세요.
"""

def get_most_frequent_answer(prompt,iterations=10):
    answers = defaultdict(int)
    for idx in range(iterations):
        context = [{'role':'user', 'content':prompt}]
        response = client.chat.completions.create(
            # model = 'gpt-3.5-turbo',  # 다양한 답이 생성, 정답을 못 맞춤
            model = 'gpt-4o-mini',      # 정답이 거의 같음 : 27분, 빈도표: [('27분', 8), ('26분', 1), ('24분', 1)]
            messages = context,
            temperature = 0.2
        )
        response_content = response.choices[0].message.content

        print(f"\n{idx+1}번째 샘플:")
        print(response_content)
         
        match = re.search(r"정답: (\d+분)", response_content)
        
        if match:
            parsed_answer = match.group(1)
            answers[parsed_answer] += 1
        print('-'*60)  

    # 빈도가 가장 높은 답을 선택   
    sorted_answers = sorted(answers.items(),key=lambda x:x[1],reverse=True)
    print(f"\n빈도표: {sorted_answers}")
    most_frequent_answer = sorted_answers[0]
    return most_frequent_answer
  
most_frequent_answer = get_most_frequent_answer(prompt) 
print('최빈값:',most_frequent_answer)


1번째 샘플:
생존자들의 강을 건너는 최적의 방법은 다음과 같습니다:

1. 3분과 4분이 함께 강을 건넙니다. (4분 소요)
2. 3분이 돌아옵니다. (3분 소요)
3. 7분과 12분이 함께 강을 건넙니다. (12분 소요)
4. 4분이 돌아옵니다. (4분 소요)
5. 3분과 4분이 다시 함께 강을 건넙니다. (4분 소요)

이 과정을 통해 각 생존자가 강을 건너는 총 소요 시간은 다음과 같습니다:

4 + 3 + 12 + 4 + 4 = 27분

따라서, 정답은 다음과 같습니다:

정답: 27분
------------------------------------------------------------

2번째 샘플:
생존자들의 강을 건너는 최적의 방법은 다음과 같습니다:

1. 3분과 4분이 함께 강을 건넙니다. (4분 소요)
2. 3분이 돌아옵니다. (3분 소요)
3. 7분과 12분이 함께 강을 건넙니다. (12분 소요)
4. 4분이 돌아옵니다. (4분 소요)
5. 3분과 4분이 다시 함께 강을 건넙니다. (4분 소요)

이 과정을 통해 각 생존자가 강을 건너는 총 소요 시간은 다음과 같습니다:

4 + 3 + 12 + 4 + 4 = 27분

따라서, 총 소요 시간은 '정답: 27분'입니다.
------------------------------------------------------------

3번째 샘플:
생존자들의 강을 건너는 시간을 최소화하기 위해 다음과 같은 전략을 사용할 수 있습니다.

1. 3분과 4분이 함께 강을 건넙니다. (4분 소요)
2. 3분이 돌아옵니다. (3분 소요)
3. 7분과 12분이 함께 강을 건넙니다. (12분 소요)
4. 4분이 돌아옵니다. (4분 소요)
5. 3분과 4분이 다시 함께 강을 건넙니다. (4분 소요)

이 과정을 통해 각 단계의 소요 시간을 합산하면 다음과 같습니다:

- 1단계: 4분
- 2단계: 3분
- 3단계: 12분
- 4단계: 4분
- 5단계: 4분

총 소요 시간 = 4 + 3 + 12 + 4

In [73]:
# import re
# data = '''
# 이 과정을 통해 모든 생존자가 강을 건너는 데 걸리는 총 시간은 39분입니다.
# 정답: 39분
# '''
# match = re.search(r"(정답): (\d+분)", data)
# parsed_answer = match.group(2)
# print(parsed_answer)

In [72]:
# data = [('a',30),('b',15),('c',120)]
# result = sorted(data,key=lambda x:x[1],reverse=True)
# print(result)

#### <br>
## Prompt Chaining (프롬프트 연쇄)
: 복잡한 문제를 해결하기 위해 여러 단계를 거쳐 연속적으로 프롬프트를 작성하는 방법 <br>
https://devocean.sk.com/blog/techBoardDetail.do?ID=166026&boardType=techBlog